<div style="font-family: 'Trebuchet MS', sans-serif; font-size: 36px; text-align: center; color: #FFD700; background-color: #1E1E2F; padding: 30px; border: 3px solid #FFD700; border-radius: 15px; box-shadow: 0 6px 15px rgba(0, 0, 0, 0.4); margin-bottom: 25px;">
    <strong>EigenFace Recognizer using OpenCV</strong>
</div>


<div style="font-family: 'Trebuchet MS', sans-serif; font-size: 20px; text-align: left; color: #D3D3D3; background-color: #1E1E2F; padding: 25px; border: 3px solid #FFD700; border-radius: 15px; box-shadow: 0 6px 15px rgba(0, 0, 0, 0.4); margin-bottom: 25px;">
    <strong style="color: #FFD700; font-size: 24px;">What is EigenFace Recognition?</strong>
    <ul style="color: #D3D3D3; font-size: 18px; margin-top: 10px;">
        <li>It is a face recognition technique based on Principal Component Analysis (PCA).</li>
        <li>It reduces the dimensionality of face images by extracting important features (Eigenfaces).</li>
        <li>The model then compares faces based on these features instead of pixel-by-pixel comparison.</li>
    </ul>
</div>


In [1]:
import cv2
print(cv2.__version__)
print(dir(cv2.face))  # Check if 'face' module is available


4.11.0
['BIF', 'BIF_create', 'BasicFaceRecognizer', 'EigenFaceRecognizer', 'EigenFaceRecognizer_create', 'FaceRecognizer', 'Facemark', 'FacemarkAAM', 'FacemarkKazemi', 'FacemarkLBF', 'FacemarkTrain', 'FisherFaceRecognizer', 'FisherFaceRecognizer_create', 'LBPHFaceRecognizer', 'LBPHFaceRecognizer_create', 'MACE', 'MACE_create', 'MACE_load', 'PredictCollector', 'StandardCollector', 'StandardCollector_create', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_native', 'createFacemarkAAM', 'createFacemarkKazemi', 'createFacemarkLBF', 'drawFacemarks', 'getFacesHAAR', 'loadDatasetList', 'loadFacePoints', 'loadTrainingData']


<div style="font-family: 'Trebuchet MS', sans-serif; font-size: 36px; text-align: center; color: #FFD700; background-color: #1E1E2F; padding: 30px; border: 3px solid #FFD700; border-radius: 15px; box-shadow: 0 6px 15px rgba(0, 0, 0, 0.4); margin-bottom: 25px;">
    <strong>Train the recognizer</strong>
</div>


In [4]:
import cv2
import os
import numpy as np

# Initialize the EigenFace Recognizer
recognizer = cv2.face.EigenFaceRecognizer_create()

# Define dataset path
dataset_path = "dataset/"  # Ensure this is the correct path

# Lists to store images and labels
faces = []
labels = []
label_dict = {}  # To store label-name mapping
label_id = 0

# Read images and assign labels
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)

    if os.path.isdir(person_path):  # Ensure it's a directory
        label_dict[label_id] = person_name  # Map label to name

        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale

            if img is None:
                continue  # Skip if image not loaded

            img = cv2.resize(img, (200, 200))  # Resize to standard size
            faces.append(np.array(img, dtype=np.uint8))  # Convert to NumPy array
            labels.append(label_id)

        label_id += 1  # Increment label ID

# Convert lists to NumPy arrays (correct type)
faces = np.array(faces, dtype=np.uint8)  
labels = np.array(labels, dtype=np.int32)

# Train the recognizer
recognizer.train(faces, labels)
recognizer.save("eigenface_model.yml")  # Save the trained model
print("✅ Model trained and saved as 'eigenface_model.yml'")


✅ Model trained and saved as 'eigenface_model.yml'


<div style="font-family: 'Trebuchet MS', sans-serif; font-size: 36px; text-align: center; color: #FFD700; background-color: #1E1E2F; padding: 30px; border: 3px solid #FFD700; border-radius: 15px; box-shadow: 0 6px 15px rgba(0, 0, 0, 0.4); margin-bottom: 25px;">
    <strong>Identification or Testing</strong>
</div>


In [5]:
import cv2

# Load trained model
recognizer = cv2.face.EigenFaceRecognizer_create()
recognizer.read("eigenface_model.yml")  # Load the trained model

# Load Haarcascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open webcam
camera = cv2.VideoCapture(0)

if not camera.isOpened():
    print("Error: Unable to access the camera")
    exit()

print("Press 'q' to quit")

while True:
    ret, frame = camera.read()
    if not ret:
        print("Error: Failed to capture frame")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    for (x, y, w, h) in faces:
        roi = gray_frame[y:y + h, x:x + w]
        roi = cv2.resize(roi, (200, 200))  # Resize for model

        label, confidence = recognizer.predict(roi)  # Predict face
        name = label_dict.get(label, "Unknown")  # Get name from label

        # Draw rectangle around face
        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)  # Green for known, Red for unknown
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

        # Put name on face
        cv2.putText(frame, f"{name} ({int(confidence)})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()


Press 'q' to quit


# Advantages:
- ✅ Fast computation
- ✅ Works well in controlled lighting conditions
- ✅ Effective for face classification

# Disadvantages:
- ❌ Sensitive to lighting changes
- ❌ Less robust compared to modern deep learning-based face recognition methods